In [128]:
import pandas as pd
import numpy as np
from typing import Dict, Tuple
from os import PathLike, mkdir
fpath = "sample_sheet.csv"

In [129]:
class Ballot(object):
    
    def __init__(self, votes: list):
        self.votes = [0] * len(votes)
        for i, v in enumerate(votes):
            self.votes[int(v) - 1] = i
        
    def vote(self) -> int:
        return self.votes[0]
    
    def vote_n(self, n: int) -> int:
        return self.votes[n]
    
    def runoff(self, n: int):
        self.votes.remove(n)

In [130]:
def load_csv(path: PathLike) -> Tuple[Dict[int, str], np.array]:
    """Loads in a Qualtrics spreadsheet and returns its info.
    
    Accepts a path to a spreadsheet, and loads it in. It will then return
    a dictionary associating vote numbers to names, and an array with votes.
    TODO: rewrite without numpy or pandas
    """
    # reads in data, and creates array of votes
    data = np.array(pd.read_csv(path))
    votes = data[2:, 17:]
    
    # creates names dictionary
    temp_names = data[0][17:]
    names = {}
    for i, val in enumerate(temp_names):
        names[i] = val.split(" - ")[-1]
    
    return names, votes

In [131]:
def main(path):
    
    names, votes = load_csv(path)
    ballots = []
    n = len(votes)
    
    for v in votes:
        ballots.append(Ballot(v))
        
    cast = {}
    for v in range(len(names)):
        cast[v] = 0
        
    flag = False
    
    while True:
        for b in ballots:
            cast[b.vote()] += 1
            
        min_opt = None
        min_votes = n 
        for option in cast:
            votes = cast[option]
            if votes > n/2:
                flag = True
                break
            elif votes < min_votes:
                min_opt = option
                min_votes = votes
        
        if flag: break
        
        for b in ballots:
            b.runoff(min_opt)
        cast.pop(min_opt)
        for i in cast:
            cast[i] = 0
        
    print("selected", names[option])

In [132]:
main(fpath)

selected Once On This Island
